### Zillow Property Value Prediction — Phase 2: Feature Engineering

In [108]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### **1. Load Clean Dataset**

In [73]:
df = pd.read_csv("../data/processed/zillow_cleaned.csv")

print("Shape:", df.shape)
print(df.info())
print(df.describe().T)

Shape: (76723, 26)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76723 entries, 0 to 76722
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   airconditioningtypeid         76723 non-null  float64
 1   bathroomcnt                   76723 non-null  float64
 2   bedroomcnt                    76723 non-null  float64
 3   buildingqualitytypeid         76723 non-null  float64
 4   calculatedfinishedsquarefeet  76723 non-null  float64
 5   fips                          76723 non-null  float64
 6   fireplacecnt                  76723 non-null  float64
 7   garagecarcnt                  76723 non-null  float64
 8   garagetotalsqft               76723 non-null  float64
 9   heatingorsystemtypeid         76723 non-null  float64
 10  latitude                      76723 non-null  float64
 11  longitude                     76723 non-null  float64
 12  lotsizesquarefeet             76723 non-n

### **2. Domain-Driven Features**

##### Prices per Square Foot, Age of Home, Bath per Bed

In [74]:
df['price_per_sqft'] = df['taxvaluedollarcnt'] / df['calculatedfinishedsquarefeet']
df["age_of_home"] = 2025 - df["yearbuilt"]
df["bath_per_bed"] = df["bathroomcnt"] / df["bedroomcnt"]


##### Rooms per Square Foot


In [ ]:
df["rooms_per_sqft"] = df["roomcnt"] / (df["calculatedfinishedsquarefeet"] + 1e-5)

# Count how many rows have rooms_per_sqft = 0
zero_count = (df["rooms_per_sqft"] == 0).sum()
total_rows = len(df)
print(f"Rows with rooms_per_sqft = 0: {zero_count}")

# Check unique values and their counts
print(df["roomcnt"].value_counts().sort_index())
print(df["bathroomcnt"].value_counts().sort_index())
print(df["bedroomcnt"].value_counts().sort_index())

df["roomcnt_fixed"] = np.where(
    df["roomcnt"] > 0,
    df["roomcnt"], 
    df["bedroomcnt"] + df["bathroomcnt"] + 1)

df["rooms_per_sqft"] = df["roomcnt_fixed"] / (df["calculatedfinishedsquarefeet"] + 1e-5)

df = df.drop(columns=["roomcnt"])

Rows with rooms_per_sqft = 0: 59131
roomcnt
0.0     59131
1.0         1
2.0         9
3.0       212
4.0      1422
5.0      3022
6.0      4519
7.0      3842
8.0      2872
9.0      1194
10.0      340
11.0      108
12.0       34
13.0       10
14.0        6
15.0        1
Name: count, dtype: int64
bathroomcnt
1.0     12739
1.5      1419
2.0     31537
2.5      6606
3.0     17346
3.5      1036
4.0      3353
4.5       696
5.0      1025
5.5       224
6.0       419
6.5        47
7.0       114
7.5        16
8.0       108
8.5         3
9.0        23
10.0        7
11.0        3
13.0        1
18.0        1
Name: count, dtype: int64
bedroomcnt
1.0      3387
2.0     19221
3.0     30428
4.0     17546
5.0      4550
6.0      1000
7.0       208
8.0       253
9.0        70
10.0       31
11.0        9
12.0       16
13.0        2
14.0        1
16.0        1
Name: count, dtype: int64


Room Count vs Bedrooms/Bathrooms

- `roomcnt` has 77% zeros → unrealistic for properties.  
- `bedroomcnt` and `bathroomcnt` are valid (≥0).  
- For rows where `roomcnt > 0`, it is always ≥ bedrooms + bathrooms, which makes sense since room count includes kitchens, living rooms, etc.  
- Therefore, I reconstructed a new `roomcnt_fixed` feature for roomcnt = 0 with bathroom + bedrooms + 1 (kitchen/other)


##### Garage Total Square Foot

In [77]:
print(df["garagetotalsqft"].value_counts().sort_index())
print(df["garagecarcnt"].value_counts().sort_index())

inconsistent = df[(df["garagetotalsqft"] == 0) & (df["garagecarcnt"] > 0)]
print("Rows with sqft=0 but cars>0:", inconsistent.shape[0])

# Case 1: Keep rows where both are 0 (real no garage)
# Case 2: Set sqft=0 to NaN if cars > 0
df.loc[(df["garagetotalsqft"] == 0) & (df["garagecarcnt"] > 0), "garagetotalsqft"] = np.nan

# Impute missing garage sqft by median for each car count
df["garagetotalsqft"] = df.groupby("garagecarcnt")["garagetotalsqft"]\
                          .transform(lambda x: x.fillna(x.median()))

print("Remaining missing values in garagetotalsqft:", df["garagetotalsqft"].isna().sum())

df["garage_sqft_ratio"] = df["garagetotalsqft"] / df["calculatedfinishedsquarefeet"]


garagetotalsqft
0.0       7530
136.0        1
144.0        1
150.0        2
152.0        2
          ... 
2901.0       1
3115.0       1
3233.0       1
3791.0       1
4251.0       1
Name: count, Length: 838, dtype: int64
garagecarcnt
0.0        79
1.0      5672
2.0     70013
3.0       631
4.0       255
5.0        45
6.0        14
7.0         4
8.0         2
9.0         2
10.0        3
11.0        1
13.0        1
14.0        1
Name: count, dtype: int64
Rows with sqft=0 but cars>0: 7451
Remaining missing values in garagetotalsqft: 0


During feature engineering, I noticed inconsistencies between `garagecarcnt` and `garagetotalsqft`: 
- **7,530 rows** had `garagetotalsqft = 0`.
- Of these, **7,451 rows also had `garagecarcnt > 0`**, which is unrealistic (a garage for cars cannot have 0 square feet).
- This indicates that many zeros in `garagetotalsqft` are **missing values** recorded as `0`, not true "no garage" cases.

Cleaning Strategy:

1. If both `garagecarcnt = 0` and `garagetotalsqft = 0` → kept as valid "no garage".
2. If `garagecarcnt > 0` but `garagetotalsqft = 0` → treated as missing.
3. Imputed missing `garagetotalsqft` using the **median garage size for each `garagecarcnt` group**  
   (e.g., ~200 sqft for 1-car garages, ~400 sqft for 2-car garages, etc.).

### **3. Encoding Categorical Features**

In [102]:
# 1. unitcnt → binary flag
df["multi_unit"] = (df["unitcnt"] > 1).astype(int)

# 2. OHE low-cardinality categorical features
low_card = ["airconditioningtypeid","heatingorsystemtypeid","fips","regionidcounty"]
df = pd.get_dummies(df, columns=low_card, drop_first=True)

# 3. propertycountylandusecode → Top-K
top_landuse = df["propertycountylandusecode"].value_counts().nlargest(15).index
df["propertycountylanduse_top"] = np.where(df["propertycountylandusecode"].isin(top_landuse),
                                           df["propertycountylandusecode"], "other")
df = pd.get_dummies(df, columns=["propertycountylanduse_top"], drop_first=True)

# 4. propertylandusetypeid → Top-K
top_landusetype = df["propertylandusetypeid"].value_counts().nlargest(5).index
df["propertylandusetype_top"] = np.where(df["propertylandusetypeid"].isin(top_landusetype),
                                         df["propertylandusetypeid"], "other")
df = pd.get_dummies(df, columns=["propertylandusetype_top"], drop_first=True)

# 5. regionidcity / regionidzip / regionidneighborhood → Top-K
for col, k in [("regionidcity", 50),("regionidzip", 50),("regionidneighborhood", 50)]:
    top_vals = df[col].value_counts().nlargest(k).index
    df[col+"_top"] = np.where(df[col].isin(top_vals), df[col], -1)
    df = pd.get_dummies(df, columns=[col+"_top"], drop_first=True)

# 6. Drop super messy zoning if you want (or keep top categories only)
df1 = df.drop(columns=["propertyzoningdesc"])


In [ ]:
# Check for encoded col names
for keyword in ["county", "zip", "city", "neighborhood", 
                "propertycountylanduse", "propertylandusetype"]:
    cols = [c for c in df1.columns if keyword in c]
    print(f"{keyword}: {len(cols)} columns")


county: 18 columns
zip: 51 columns
city: 51 columns
neighborhood: 51 columns
propertycountylanduse: 16 columns
propertylandusetype: 6 columns


- **Low-cardinality categorical features**  
  - `airconditioningtypeid`, `heatingorsystemtypeid`, `fips`, `regionidcounty` → one-hot encoded.  
- **Ordinal feature**  
  - `buildingqualitytypeid` retained as numeric (ordinal scale 1–12).  
- **Unit count**  
  - Converted `unitcnt` into binary flag `multi_unit` (1 = multi-unit, 0 = single-unit).  
- **Land use**  
  - `propertycountylandusecode` → encoded with Top-15 most frequent categories, others grouped as `"other"`.  
  - `propertylandusetypeid` → encoded with Top-5 most frequent categories, others grouped as `"other"`.  
- **High-cardinality location features**  
  - `regionidzip` → Top-50 ZIPs, rest grouped as `"other"`.  
  - `regionidcity` → Top-50 cities, rest grouped as `"other"`.  
  - `regionidneighborhood` → Top-50 neighborhoods, rest grouped as `"other"`.  
- **Property zoning**  
  - Dropped `propertyzoningdesc` due to extreme cardinality (1,800+ unique codes).  

- Total number of columns after encoding: **~215** (from ~30 original + engineered features).  